In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
# file_path = './updated_doctor_data.csv'
file_path1='./doctors_data_multireward.csv'
# data = pd.read_csv(file_path)
data1=pd.read_csv(file_path1)
data1.head(), data1.columns


(        Doctor Speciality          Doctor Name  Ratings  Experience  \
 0  Allergy and Immunology       Dr. Aaron Haas     3.42        4.69   
 1  Allergy and Immunology     Dr. Toni Holland     3.08        3.88   
 2  Allergy and Immunology       Dr. Julie Gray     4.85        0.86   
 3  Allergy and Immunology      Dr. Kari Thomas     5.31        3.51   
 4  Allergy and Immunology  Dr. William Manning     5.38        8.58   
 
    Distance from Patient  Availability  Cost of Services  
 0                   6.44             0              3.81  
 1                   4.51             1              8.42  
 2                   7.80             1             27.32  
 3                   5.78             0             12.74  
 4                   5.28             1              1.64  ,
 Index(['Doctor Speciality', 'Doctor Name', 'Ratings', 'Experience',
        'Distance from Patient', 'Availability', 'Cost of Services'],
       dtype='object'))

In [3]:
cols_drop = ["Availability","Distance from Patient","Cost of Services"]
data1.drop(columns=cols_drop, inplace=True)
# import matplotlib.pyplot as plt


In [4]:
data1.columns

Index(['Doctor Speciality', 'Doctor Name', 'Ratings', 'Experience'], dtype='object')

In [ ]:
import numpy as np
import pandas as pd

class MultiArmedBanditExperience:
    def __init__(self, data, epsilon=0.5, ratings_weight=0.0, experience_weight=0.0):
        self.data = data
        self.epsilon = epsilon
        self.ratings_weight = ratings_weight
        self.experience_weight = experience_weight
        
        # Normalize Ratings and Experience
        self.data['Normalized Ratings'] = (self.data['Ratings'] - self.data['Ratings'].min()) / (self.data['Ratings'].max() - self.data['Ratings'].min())
        self.data['Normalized Experience'] = (self.data['Experience'] - self.data['Experience'].min()) / (self.data['Experience'].max() - self.data['Experience'].min())
        
        # Compute composite scores
        self.data['Composite Score'] = self.data['Normalized Ratings'] * self.ratings_weight + self.data['Normalized Experience'] * self.experience_weight

        # Initialize dictionaries to keep track of composite scores and counts for each doctor
        self.scores = self.data.groupby(['Doctor Speciality', 'Doctor Name'])['Composite Score'].mean().to_dict()
        self.counts = self.data.groupby(['Doctor Speciality', 'Doctor Name'])['Composite Score'].count().to_dict()

    def recommend_doctor(self, specialists):
        recommendations = {}
        for specialist in specialists:
            doctors = self.data[self.data['Doctor Speciality'] == specialist]
            if doctors.empty:
                recommendations[specialist] = "No doctors available for this specialty."
                continue

            # Sort doctors by composite score in descending order
            sorted_doctors = doctors.sort_values(by='Composite Score', ascending=False).head(5)
            top_doctors = []
            recommended_names = set()  # Set to store recommended doctor names

            for _ in range(5):  # Select top 5 doctors based on stochastic policy
                if np.random.rand() < self.epsilon:  # Explore
                    # Randomly choose a doctor
                    recommendation = doctors.sample()
                else:  # Exploit
                    max_score = -np.inf
                    for index, row in doctors.iterrows():
                        key = (specialist, row['Doctor Name'])
                        if self.scores[key] > max_score and row['Doctor Name'] not in recommended_names:
                            max_score = self.scores[key]
                            recommendation = row.to_frame().T
                            recommended_names.add(row['Doctor Name'])  # Add doctor name to set of recommended names

                doctor_name = recommendation['Doctor Name'].iloc[0]
                doctor_rating = recommendation['Ratings'].iloc[0]
                doctor_experience = recommendation['Experience'].iloc[0]
                top_doctors.append((doctor_name, doctor_rating, doctor_experience))

            recommendations[specialist] = top_doctors

        return recommendations


# Usage Example:
specialists_input = input("Enter the list of specialists you are looking for, separated by commas: ").split(',')
user_exp_weight = float(input("On a scale of 0-9, how much weight do you want to give to the doctor's experience: ")) / 10
ratings_weight = 1 - user_exp_weight


bandit_with_experience = MultiArmedBanditExperience(data1, ratings_weight=ratings_weight, experience_weight=user_exp_weight)

results = bandit_with_experience.recommend_doctor(specialists_input)
print(f"Recommended Doctors with Ratings and Experience for : {specialists_input}")
for specialist, top_doctors in results.items():
   
    print("-" * 60)
    print(f"Specilaity : {specialist}")
    print("-" * 60)
    print(f"{'Doctor Name':30s} | {'Rating':10s} | {'Experience':10s}")
    print("-" * 60)
    for doctor in top_doctors:
        print(f"{doctor[0]:30s} | {doctor[1]:10.2f} | {doctor[2]:10.2f}")
print("-" * 60)

In [ ]:
def calculate_cumulative_regret(model, data, specialists, num_trials):
    cumulative_regret = 0
    for _ in range(num_trials):
        recommendations = model.recommend_doctor(specialists)
        for specialist in specialists:
            top_doctors = recommendations[specialist]
            if type(top_doctors) == str:  # Handles the case where no doctors are available
                continue
            optimal_score = data[data['Doctor Speciality'] == specialist]['Composite Score'].max()
            optimal_rating = data[data['Composite Score'] == optimal_score]['Ratings'].max()
            optimal_experience = data[data['Composite Score'] == optimal_score]['Experience'].max()

            top_recommended_rating = top_doctors[0][1]
            top_recommended_experience = top_doctors[0][2]
           
            normalized_optimal_score = (optimal_rating - data['Ratings'].min()) / (data['Ratings'].max() - data['Ratings'].min()) + \
                                       (optimal_experience - data['Experience'].min()) / (data['Experience'].max() - data['Experience'].min())
            normalized_recommended_score = (top_recommended_rating - data['Ratings'].min()) / (data['Ratings'].max() - data['Ratings'].min()) + \
                                           (top_recommended_experience - data['Experience'].min()) / (data['Experience'].max() - data['Experience'].min())
            cumulative_regret += (normalized_optimal_score - normalized_recommended_score)

    return cumulative_regret
specialists_input = ['Dermatology', 'Cardiology'] 
num_trials = 500
cumulative_regret = calculate_cumulative_regret(bandit_with_experience, data1, specialists_input, num_trials)
print(f"Cumulative Regret after {num_trials} trials: {cumulative_regret}")


In [42]:
import numpy as np
import pandas as pd
import math

class UCB1Experience:
    def __init__(self, data, ratings_weight=0.0, experience_weight=0.0):
        self.data = data
        # Normalize Ratings and Experience
        self.data['Normalized Ratings'] = (self.data['Ratings'] - self.data['Ratings'].min()) / (self.data['Ratings'].max() - self.data['Ratings'].min())
        self.data['Normalized Experience'] = (self.data['Experience'] - self.data['Experience'].min()) / (self.data['Experience'].max() - self.data['Experience'].min())
        
        # Compute composite scores
        self.data['Composite Score'] = self.data['Normalized Ratings'] * ratings_weight + self.data['Normalized Experience'] * experience_weight

        # Initialize counts and sum_composite_scores for each doctor
        self.counts = self.data.groupby(['Doctor Speciality', 'Doctor Name']).size().to_dict()
        self.sum_composite_scores = self.data.groupby(['Doctor Speciality', 'Doctor Name'])['Composite Score'].sum().to_dict()
        self.total_counts = 0  # Total number of times any doctor has been chosen

    def recommend_doctors(self, specialists):
        recommendations = {}
        for specialist in specialists:
            doctors = self.data[self.data['Doctor Speciality'] == specialist.strip()]
            if doctors.empty:
                recommendations[specialist] = "No doctors available for this specialty."
                continue

            doctor_ucb_scores = []
            for index, row in doctors.iterrows():
                key = (specialist, row['Doctor Name'])
                if self.counts[key] == 0:
                    ucb = float('inf')  # Infinite UCB for unexplored options
                else:
                    average_composite_score = self.sum_composite_scores[key] / self.counts[key]
                    ucb = average_composite_score + math.sqrt((2 * math.log(self.total_counts + 1)) / self.counts[key])

                doctor_ucb_scores.append((row['Doctor Name'], row['Ratings'], row['Experience'], ucb))

            # Sort by UCB score descending and take the top 5
            doctor_ucb_scores.sort(key=lambda x: x[3], reverse=True)
            recommendations[specialist] = doctor_ucb_scores[:5]

            # Update model state
            for doc in doctor_ucb_scores[:5]:
                self.counts[(specialist, doc[0])] += 1
                self.sum_composite_scores[(specialist, doc[0])] += self.data[self.data['Doctor Name'] == doc[0]]['Composite Score'].iloc[0]
                self.total_counts += 1

        return recommendations

# Usage Example:
specialists_input = input("Enter the list of specialists you are looking for, separated by commas: ").split(',')
user_exp_weight = float(input("On a scale of 0-9, how much weight do you want to give to the doctor's experience: ")) / 10
ratings_weight = 1 - user_exp_weight

ucb_with_experience = UCB1Experience(data1, ratings_weight=ratings_weight, experience_weight=user_exp_weight)

results = ucb_with_experience.recommend_doctors(specialists_input)
# for specialist, top_doctors in results.items():
#     print(f"Top 5 doctors for {specialist}:")
#     for doctor in top_doctors:
#         print(f"    Doctor Name: {doctor[0]}, Rating: {doctor[1]}, Experience: {doctor[2]}")


print(f"Recommended Doctors with Ratings and Experience for : {specialists_input}")
for specialist, top_doctors in results.items():
   
    print("-" * 60)
    print(f"Specilaity : {specialist}")
    print("-" * 60)
    print(f"{'Doctor Name':30s} | {'Rating':10s} | {'Experience':10s}")
    print("-" * 60)
    for doctor in top_doctors:
        print(f"{doctor[0]:30s} | {doctor[1]:10.2f} | {doctor[2]:10f}")
print("-" * 60)

Enter the list of specialists you are looking for, separated by commas: Dermatology,Cardiology
On a scale of 0-9, how much weight do you want to give to the doctor's experience: 5
Recommended Doctors with Ratings and Experience for : ['Dermatology', 'Cardiology']
------------------------------------------------------------
Specilaity : Dermatology
------------------------------------------------------------
Doctor Name                    | Rating     | Experience
------------------------------------------------------------
Dr. Joshua Mercado             |       8.92 |  30.000000
Dr. Tiffany Miranda            |       7.13 |  30.000000
Dr. John Webb                  |       7.83 |  27.700000
Dr. Deborah Luna               |       6.94 |  30.000000
Dr. Krista Moore               |       6.64 |  29.300000
------------------------------------------------------------
Specilaity : Cardiology
------------------------------------------------------------
Doctor Name                    | Rating 

In [50]:
def calculate_cumulative_regret(model, data, specialists, num_trials):
    cumulative_regret = 0
    for _ in range(num_trials):
        recommendations = model.recommend_doctors(specialists)
        for specialist in specialists:
            top_doctors = recommendations[specialist]
            if type(top_doctors) == str:  # Handles the case where no doctors are available
                continue
            optimal_score = data[data['Doctor Speciality'] == specialist]['Composite Score'].max()
            optimal_rating = data[data['Composite Score'] == optimal_score]['Ratings'].max()
            optimal_experience = data[data['Composite Score'] == optimal_score]['Experience'].max()

            top_recommended_rating = top_doctors[0][1]
            top_recommended_experience = top_doctors[0][2]
           
            normalized_optimal_score = (optimal_rating - data['Ratings'].min()) / (data['Ratings'].max() - data['Ratings'].min()) + \
                                       (optimal_experience - data['Experience'].min()) / (data['Experience'].max() - data['Experience'].min())
            normalized_recommended_score = (top_recommended_rating - data['Ratings'].min()) / (data['Ratings'].max() - data['Ratings'].min()) + \
                                           (top_recommended_experience - data['Experience'].min()) / (data['Experience'].max() - data['Experience'].min())
            cumulative_regret += (normalized_optimal_score - normalized_recommended_score)

    return cumulative_regret
specialists_input = ['Dermatology', 'Cardiology'] 
num_trials = 500
cumulative_regret = calculate_cumulative_regret(ucb_with_experience, data1, specialists_input, num_trials)
print(f"Cumulative Regret after {num_trials} trials: {cumulative_regret}")


Cumulative Regret after 500 trials: 1082.5770000000002


In [ ]:
def calculate_coverage(model, data, specialist, num_trials):
    recommended = set()
    all_doctors = set(data[data['Doctor Speciality'] == specialist]['Doctor Name'])
    for _ in range(num_trials):
        doc_name, _, _ = model.recommend_doctor(specialist)
        recommended.add(doc_name)
    coverage = len(recommended) / len(all_doctors)
    return coverage

num_trials = 1000
specialist="Dermatologist"

coverage = calculate_coverage(bandit_with_experience, data, specialist, num_trials)
print(f"Coverage: {coverage}")


In [ ]:
def calculate_cumulative_regret(model, data, specialist, num_trials, patient_age):
    optimal_score = data[data['Doctor Speciality'] == specialist]['Composite Score'].max()
    cumulative_regret = 0
    for _ in range(num_trials):
        _, rating, _ = model.recommend_doctor(specialist, patient_age)
        current_score = (rating - data['Ratings'].min()) / (data['Ratings'].max() - data['Ratings'].min())  # Normalize current rating as done in model
        cumulative_regret += (optimal_score - current_score)
    return cumulative_regret


def calculate_average_reward(model, data, specialist, num_trials, patient_age):
    total_score = 0
    for _ in range(num_trials):
        _, rating, _ = model.recommend_doctor(specialist, patient_age)
        total_score += rating
    return total_score / num_trials

def precision_recall_at_k(model, data, specialist, k, threshold, patient_age):
    recommended_docs = []
    for _ in range(k):
        doc_name, rating, _ = model.recommend_doctor(specialist, patient_age)
        recommended_docs.append((doc_name, rating))
    relevant_docs = data[(data['Doctor Speciality'] == specialist) & (data['Ratings'] >= threshold)]
    recommended_relevant = [doc for doc, rating in recommended_docs if rating >= threshold]
    precision = len(recommended_relevant) / k
    recall = len(recommended_relevant) / len(relevant_docs)
    return precision, recall

def calculate_coverage(model, data, specialist, num_trials, patient_age):
    recommended = set()
    all_doctors = set(data[data['Doctor Speciality'] == specialist]['Doctor Name'])
    for _ in range(num_trials):
        doc_name, _, _ = model.recommend_doctor(specialist, patient_age)
        recommended.add(doc_name)
    coverage = len(recommended) / len(all_doctors)
    return coverage

num_trials = 1000
specialist = "Dermatologist"
patient_age = 65  # Example age

cumulative_regret = calculate_cumulative_regret(ucb_with_experience, data, specialist, num_trials, patient_age)
average_reward = calculate_average_reward(ucb_with_experience, data, specialist, num_trials, patient_age)
precision, recall = precision_recall_at_k(ucb_with_experience, data, specialist, 10, 7.0, patient_age)  # top 5 recommendations, threshold rating of 8.0
coverage = calculate_coverage(ucb_with_experience, data, specialist, num_trials, patient_age)

print(f"Cumulative Regret: {cumulative_regret}")
print(f"Average Reward: {average_reward}")
print(f"Precision at 5: {precision}, Recall at 5: {recall}")
print(f"Coverage: {coverage}")


In [49]:
def calculate_hit_ratio(recommendations, data, specialist):
    top_doctors = data[data['Doctor Speciality'] == specialist]
    top_doctor_names = top_doctors[top_doctors['Composite Score'] >= top_doctors['Composite Score'].quantile(0.99)]['Doctor Name'].tolist()
    
    # Extract just the doctor names from the recommendations
    recommended_doctor_names = [doc[0] for doc in recommendations]  # doc[0] is the doctor's name in each tuple
    
    hits = sum([1 for doctor_name in recommended_doctor_names if doctor_name in top_doctor_names])
    hit_ratio = hits / len(recommendations) if recommendations else 0
    return hit_ratio


# Assuming distinct_recommendations and bandit_with_experience are initialized and populated as above
hit_ratio = calculate_hit_ratio(distinct_recommendations, data, specialist_input)
print("Hit ratio:", hit_ratio)


Hit ratio: 1.0
